# Regresja logistyczna wielomianowa

Model regresji logistycznej, dla targetu z więcej niż dwiema kategoriami.

Zbiór danych: https://www.kaggle.com/datasets/atefehmirnaseri/cell-phone-price

In [1]:
# Import wymaganych bibliotek
from  sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
os.chdir('../')

In [2]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [3]:
df = pd.read_csv('data/CellPhone.csv')

## Problem biznesowy
Model służący do wyceny telefonów komórkowych (przypisanie do jednej z 4 kategorii cenowych). Model może być wykorzystywanych w obrocie telefonami używanymi. Usprawnia wycenę oraz sprawia, że jest ona dokładniejsza, przez co można szybciej znaleźć nowego nabywcę i zoptymalizować zyski.
Cel szczegółowy: zwiększenie zysków, poprzez precyzyjną wycenę wykupu.

## Data preprocessing

In [ ]:
# Nagłówek
df.head()

In [7]:
pd.set_option('display.max_columns',None)

In [ ]:
# podsumowanie zmiennych
df.describe()

In [ ]:
# Liczebności klas
df.price_range.value_counts()

Kategorie są równoliczne, więc zbiór jest zbilansowany, nie musimy się więc tym martwić.
### Analiza relacji z targetem

In [ ]:
df.info()

In [12]:
# Lista potencjalnych X
x_names = df.columns[:-1]

In [13]:
# target name
target_name = 'price_range'

In [ ]:
for i in x_names:
    print("#########################")
    print(f'Start analysis for {i}')
    print(df.groupby(target_name).describe()[i])
    fig, ax = plt.subplots(ncols=2, nrows =1 )
    sns.kdeplot(ax = ax[0], data = df, x= i, hue = target_name, fill = True, common_norm= False)
    sns.boxplot(ax = ax[1], data= df, y = i, hue = target_name, fill = True)
    plt.show()
    print(f'End analysis for {i}')
    print("#########################")

Największe różnicowanie widzimy dla zmiennej 'ram'. Kilka zmiennych wykazują pewnie różnice, szczególnie pomiędzy kategorią 0 i 3 np. 'battery_power', 'talk_time' oraz 'px_width' . 
Część zmiennych ma skończoną liczbę kategorii, więc potraktujemy je jako zmienne kategoryczne


In [20]:
# Stworzenie słownika z liczbą unikalnyc wartości
unique_numbers = {i: len(df[i].unique()) for i in x_names}

In [ ]:
unique_numbers

In [65]:
# Zmienne z liczbą kategorii <=10
low_number_of_cat = pd.Series(unique_numbers.values())<=10

In [ ]:
# Wyciągnięcie indeksów
idx = list(low_number_of_cat[low_number_of_cat==True].index)
idx

In [64]:
# Nazwy zmiennych
x_names_cat = pd.Series(unique_numbers.keys())[idx]

In [66]:
df['denominator'] = 1

In [ ]:
for i in x_names_cat:
    print(f'Analysis for {i}')
    pivot_summary = pd.pivot_table(data = df, index = i, columns = target_name, aggfunc='sum', margins=True)['denominator']
    df_percentage = round(pivot_summary / pivot_summary.loc['All',:],2).reset_index()
    print(df_percentage)
    print(f'End analysis for {i} \n')

Zmienne mają bardzo zbliżony udział poszczególnych kategorii cen w danych wartościach. Jedynie możemy wytypować jedną zmienną: 'four_g'.

In [76]:
x_names_final = ['ram','battery_power', 'talk_time' , 'px_width', 'four_g']

### VIF index
Do sprawdzenia współliniowości można użyć indeksu VIF. Bada on wielowymiarową korelację pomiędzy zmiennymi.
Ogólnie VIF powyżej wartości 5 sugeruje wysoką współliniowość i potrzebę usunięcia zmiennej.

Więcej informacji: https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/


In [ ]:
#pip install statsmodels

In [78]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
for i in range(len(x_names_final)):
    vif = variance_inflation_factor(df[x_names_final],i)
    print(f'{x_names_final[i]}: {vif}')

In [105]:
x_names_final = ['ram', 'talk_time' , 'px_width', 'four_g']

In [ ]:
for i in range(len(x_names_final)):
    vif = variance_inflation_factor(df[x_names_final],i)
    print(f'{x_names_final[i]}: {vif}')

## Modelowanie

In [88]:
from sklearn.model_selection import train_test_split

In [106]:
# definicja X i y
X = df.loc[:,x_names_final]
y = df.loc[:,target_name]

In [107]:
train_x, test_x, train_y, test_y = train_test_split(X,y,test_size=0.4, random_state=123)

In [109]:
# Stworzenie objektu + estymacja
model_1 = LogisticRegression(max_iter=20000).fit(train_x,train_y)

In [110]:
# predykcje
train_pred = model_1.predict(train_x)
test_pred = model_1.predict(test_x)

In [ ]:
train_pred

In [ ]:
# Prawdopodobieństwo
train_pred_proba = model_1.predict_proba(train_x)
train_pred_proba

In [114]:
from sklearn.metrics import accuracy_score

In [ ]:
# Accuracy train
accuracy_score(train_y, train_pred)

In [ ]:
# Accuracy test
accuracy_score(test_y, test_pred)